In [2]:
%pip install experta python-constraint

  Using cached experta-1.9.4-py3-none-any.whl.metadata (5.0 kB)
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for frozendict: filename=frozendict-1.2-py3-none-any.whl size=3217 sha256=1115711baec83ec29ed006b945de34d2caa0c7f462cd215909f1a2a1455c5b06
  Stored in directory: /Users/yixiang/Library/Caches/pip/wheels/f6/ff/aa/750fec7bf9618d87b53572def5abf3e098f853cc5ab4147656
  Created wheel for python-constraint: filename=python_constraint-1.4.0-py2.py3-none-any.whl size=24106 sha256=b0e243c1490e2f41e30e79b213d8bb82a25debe7263ba8de754f4f4ef309feb4
  Stored in directory: /Users/yixiang/Library/Caches/pip/wheels/c1/d2/3d/082849b61a9c6de02d4a7c8a402c224640f08d8a971307b92b
Successfully built frozendict python-constraint
  Attempting uninstall: frozendict
    

In [3]:
%pip install --upgrade frozendict

  Using cached frozendict-2.4.6-py312-none-any.whl.metadata (23 kB)
Using cached frozendict-2.4.6-py312-none-any.whl (16 kB)
  Attempting uninstall: frozendict
    Found existing installation: frozendict 1.2
    Uninstalling frozendict-1.2:
      Successfully uninstalled frozendict-1.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
experta 1.9.4 requires frozendict==1.2, but you have frozendict 2.4.6 which is incompatible.
Note: you may need to restart the kernel to use updated packages.


#### Decision Tree Modelling (Working adults)

##### Inputs:

- Age
- Planning to buy a home?
- Still repaying home loans?
- Have kids?
- Supporting aged parents?
- Monthly take-home pay
  - IF don't know THEN ask for gross salary AND whether there is CPF contribution
- Estimated current expenses on needs / wants / insurance / investment
- Estimated current emergency funds

##### Recommended Actions:

- Needs Analysis:
  - Transportation
  - Food
  - Utility bill
  - Home loan payment / Rent expense
  - Insurance expense
  - Medical expense
  - Others
  - IF transport expenditure > $150 AND (needs expenditure over budget OR too little savings): RECOMMEND taking public transport instead
  - IF utility bill > 300: RECOMMEND electricity/water saving methods or switching utilities provider 
  - IF insurance expenditure > 15% of take-home salary: REDUCE insurance expenditure
  - IF needs expenditure > 50% of take-home salary: REDUCE spending on needs
    - IF food expense > 15% of take-home salary: REDUCE spending on food
    - IF housing expense > 35% of take-home salary: RECOMMEND finding alternatives
- Wants Analysis:
  - IF wants expenditure > 30% of take-home salary: REDUCE spending on wants
- Savings Analysis:
  - Investments
  - Savings in bank
  - IF investment expenditure < 10% of take-home salary: 
    - IF age < 65: INVEST in long-term investment plans such as ETFs for retirement
    - IF have kids: INVEST in short-term investment plans for kids' tertiary education
    - IF planning to buy a home: INVEST in short-term investment plans for home loan repayment
  - IF monthly savings < 20% of take-home salary: INCREASE savings by REDUCING spending on wants or needs
- Emergency Funds Analysis:
  - IF emergency funds < 3 months of current expenses: INCREASE savings
  - ELSE IF emergency funds < 6 months of current expenses (needs + wants): RECOMMENDED to increase savings


In [11]:
# EXAMPLE USAGE OF EXPERTA

from experta import KnowledgeEngine, Rule, Fact, MATCH, TEST

# Define a Fact subclass for budget-related data
class BudgetFact(Fact):
    # You can define expected fields here if you want to enforce a schema.
    needs_expenditure = float
    take_home_income = float

# Create a KnowledgeEngine subclass with rules
class BudgetAdvisor(KnowledgeEngine):

    @Rule(BudgetFact(needs_expenditure=MATCH.needs, take_home_income=MATCH.income),
          TEST(lambda needs, income: needs > 0.5 * income))
    def high_needs_expenditure(self):
        print("Your needs expenditure is over 50% of your take-home income. Consider reducing spending on essential needs.")

    @Rule(BudgetFact(needs_expenditure=MATCH.needs, take_home_income=MATCH.income),
          TEST(lambda needs, income: needs <= 0.5 * income))
    def acceptable_needs_expenditure(self):
        print("Your needs expenditure is within an acceptable range.")

# Using the engine
engine = BudgetAdvisor()
engine.reset()  # Prepare the engine for new facts
# Declare a fact: For example, needs expenditure is 3000 SGD and take-home income is 5000 SGD.
engine.declare(BudgetFact(needs_expenditure=3000, take_home_income=5000))
engine.run()  # Process rules and execute matching ones

Your needs expenditure is over 50% of your take-home income. Consider reducing spending on essential needs.


In [1]:
from experta import KnowledgeEngine, Fact, Rule, MATCH, TEST

class BudgetInfo(Fact):
    """Holds the user's budget-related information."""
    # Basic personal details
    age = int
    planning_to_buy_home = bool
    repaying_home_loans = bool
    have_kids = bool
    supporting_aged_parents = bool
    monthly_take_home = float

    # Expenses and savings details
    transport_expenditure = float
    needs_expenditure = float
    wants_expenditure = float
    insurance_expenditure = float
    food_expenditure = float
    housing_expenditure = float
    utility_bill = float
    investment_expenditure = float
    monthly_savings = float
    emergency_funds = float

class BudgetAdvisor(KnowledgeEngine):
    # ---------------- Needs Analysis Rules ----------------
    @Rule(BudgetInfo(transport_expenditure=MATCH.transport,
                     needs_expenditure=MATCH.needs,
                     monthly_take_home=MATCH.take_home,
                     monthly_savings=MATCH.savings),
          TEST(lambda transport, needs, take_home, savings: 
               transport > 150 and (needs > 0.5 * take_home or savings < 0.2 * take_home)))
    def rule_public_transport(self, transport, needs, take_home, savings):
        print("Your transport spendings are above average. Consider taking public transport instead of using private options.")

    @Rule(BudgetInfo(utility_bill=MATCH.utility),
          TEST(lambda utility: utility > 200))
    def rule_switch_utilities(self, utility):
        print("To conserve utilities and reduce costs, consider using energy-efficient appliances, " \
        "turning off lights when leaving a room, and taking shorter showers. " \
        "You may also consider switching your utilities provider to reduce bills.")

    @Rule(BudgetInfo(insurance_expenditure=MATCH.insurance, monthly_take_home=MATCH.take_home),
          TEST(lambda insurance, take_home: insurance > 0.15 * take_home))
    def rule_reduce_insurance(self, insurance, take_home):
        print("Your spending on insurance is too high! " \
        "The recommended spending on insurance is maximum 15% of your take home-salary.")

    @Rule(BudgetInfo(needs_expenditure=MATCH.needs, monthly_take_home=MATCH.take_home),
          TEST(lambda needs, take_home: needs > 0.5 * take_home))
    def rule_reduce_needs(self, needs, take_home):
        print("Your spending on necessities are too high! " \
        "The recommended spending on needs is maximum 50% of your take-home salary.")

    @Rule(BudgetInfo(food_expenditure=MATCH.food, monthly_take_home=MATCH.take_home),
          TEST(lambda food, take_home: food > 0.15 * take_home))
    def rule_reduce_food(self, food, take_home):
        print("Your spending on food are too high! " \
        "The recommended spending on food is maximum 15% of your take-home salary.")

    @Rule(BudgetInfo(housing_expenditure=MATCH.housing, monthly_take_home=MATCH.take_home),
          TEST(lambda housing, take_home: housing > 0.35 * take_home))
    def rule_housing_alternatives(self, housing, take_home):
        print("You may want to consider looking for alternative housing options " \
        "as housing costs are over 35% of your take-home salary.")

    # ---------------- Wants Analysis Rule ----------------
    @Rule(BudgetInfo(wants_expenditure=MATCH.wants, monthly_take_home=MATCH.take_home),
          TEST(lambda wants, take_home: wants > 0.3 * take_home))
    def rule_reduce_wants(self, wants, take_home):
        print("Your spending on wants are too high! " \
        "The recommended spending on wants is maximum 30% of your take-home salary.")

    # ---------------- Savings Analysis Rules ----------------
    @Rule(BudgetInfo(investment_expenditure=MATCH.invest,
                     monthly_take_home=MATCH.take_home,
                     age=MATCH.age,
                     have_kids=MATCH.kids,
                     planning_to_buy_home=MATCH.buy_home),
          TEST(lambda invest, take_home: invest < 0.1 * take_home))
    def rule_investment_recommendations(self, age, kids, buy_home):
        if age < 65:
            print("Consider investing in long-term plans (e.g. ETFs) for retirement.")
        if kids:
            print("Consider short-term investment plans for your kids' tertiary education.")
        if buy_home:
            print("Consider short-term investment plans to help with home loan repayment.")

    @Rule(BudgetInfo(monthly_savings=MATCH.savings, monthly_take_home=MATCH.take_home),
          TEST(lambda savings, take_home: savings < 0.2 * take_home))
    def rule_increase_savings(self, savings, take_home):
        print("Your savings are too low! You are recommended to save 20% of your take-home salary. " \
        "You can increase your monthly savings by reducing spending on needs or wants.")

    # ---------------- Emergency Funds Analysis Rules ----------------
    @Rule(BudgetInfo(emergency_funds=MATCH.funds, needs_expenditure=MATCH.needs, wants_expenditure=MATCH.wants),
          TEST(lambda funds, needs, wants: funds < 3 * (needs + wants)))
    def rule_emergency_funds_low(self, funds, needs, wants):
        print("Financial experts recommend maintaining an emergency fund covering 3-6 months of your monthly expenses. " \
        "Currently, your savings fall short of the 3-month minimum. " \
        "To avoid financial stress during unexpected events, please consider increasing your savings as soon as possible.")

    @Rule(BudgetInfo(emergency_funds=MATCH.funds, needs_expenditure=MATCH.needs, wants_expenditure=MATCH.wants),
          TEST(lambda funds, needs, wants: funds > 3 * (needs + wants) and funds < 6 * (needs + wants)))
    def rule_emergency_funds_medium(self, funds, needs, wants):
        print("Financial experts recommend maintaining an emergency fund covering 3-6 months of your monthly expenses. " \
        "Build up your emergency funds to cover at least 6 months of your current expenses.")

if __name__ == "__main__":
    engine = BudgetAdvisor()
    engine.reset()

    # Example inputs – these values could be gathered interactively or via a user interface.
    engine.declare(BudgetInfo(
        age=30,
        planning_to_buy_home=True,
        repaying_home_loans=False,
        have_kids=True,
        supporting_aged_parents=False,
        monthly_take_home=4000,
        transport_expenditure=200,
        needs_expenditure=2500,
        wants_expenditure=1500,
        insurance_expenditure=700,
        food_expenditure=700,
        housing_expenditure=1000,
        utility_bill=250,
        investment_expenditure=300,
        monthly_savings=800,
        emergency_funds=8000
    ))

    engine.run()


To conserve utilities and reduce costs, consider using energy-efficient appliances, turning off lights when leaving a room, and taking shorter showers. You may also consider switching your utilities provider to reduce bills.
Financial experts recommend maintaining an emergency fund covering 3-6 months of your monthly expenses. Currently, your savings fall short of the 3-month minimum. To avoid financial stress during unexpected events, please consider increasing your savings as soon as possible.
Your spending on insurance is too high! The recommended spending on insurance is maximum 15% of your take home-salary.
Your spending on food are too high! The recommended spending on food is maximum 15% of your take-home salary.
Your spending on wants are too high! The recommended spending on wants is maximum 30% of your take-home salary.
Consider investing in long-term plans (e.g. ETFs) for retirement.
Consider short-term investment plans for your kids' tertiary education.
Consider short-term 

In [4]:
from constraint import Problem

def setup_csp(budget):
    """
    Sets up a CSP problem to allocate income among various expense categories.
    :param budget: A dictionary with key 'monthly_take_home' that holds an integer amount.
    :return: List of feasible solutions.
    """
    income = int(budget["monthly_take_home"])
    problem = Problem()
    
    # Define decision variables with a domain from 0 to income (in whole dollars)
    expense_categories = ["savings", "housing", "food", "insurance", "other"]
    for category in expense_categories:
        problem.addVariable(category, range(0, income + 1))
    
    # Add constraint: Sum of all allocations must equal the income.
    def total_allocation(*allocations):
        return sum(allocations) == income
    problem.addConstraint(total_allocation, expense_categories)
    
    # Add ratio constraints based on recommendations:
    # 1. Savings should be at least 20% of income.
    problem.addConstraint(lambda s: s >= 0.2 * income, ("savings",))
    
    # 2. Housing should be no more than 30% of income.
    problem.addConstraint(lambda h: h <= 0.3 * income, ("housing",))
    
    # 3. Food should be no more than 15% of income.
    problem.addConstraint(lambda f: f <= 0.15 * income, ("food",))
    
    # 4. Insurance should be no more than 15% of income.
    problem.addConstraint(lambda i: i <= 0.15 * income, ("insurance",))
    
    # Optionally, other policies (like CPF or debt service constraints) can be added as more constraints.
    
    solutions = problem.getSolutions()
    return solutions

# Example usage:
if __name__ == "__main__":
    budget_example = {"monthly_take_home": 4000}
    solutions = setup_csp(budget_example)
    
    # For demonstration, show the first solution found (if one exists)
    if solutions:
        print("Number of feasible allocation solutions found:", len(solutions))
        print("Example solution:")
        print(solutions[0])
    else:
        print("No feasible allocation found with the given constraints.")


KeyboardInterrupt: 

In [3]:
from constraint import Problem
from typing import Dict, Any

def setup_csp_with_dynamic_constraints(budget: Dict[str, Any], custom_constraints: Dict[str, float]):
    """
    Sets up a CSP problem to allocate income among expense categories with some constraints dynamically defined by the user.
    
    Args:
      budget: A dictionary with 'monthly_take_home' as a key.
      custom_constraints: A dictionary for dynamic constraints.
          e.g., {"savings_min": 0.3, "food_max": 0.12} meaning 
            - savings should be at least 30% of income
            - food expenditure should be at most 12% of income
    Returns:
      A list of feasible solutions.
    """
    income = int(budget["monthly_take_home"])
    problem = Problem()
    
    # Define expense categories
    expense_categories = ["savings", "housing", "food", "insurance", "other"]
    for category in expense_categories:
        # We use the full range [0, income] assuming whole-dollar increments.
        problem.addVariable(category, range(0, income + 1, 50))
    
    # Constraint: The sum of all allocations must equal the monthly income
    def total_allocation(*allocations):
        return sum(allocations) == income
    problem.addConstraint(total_allocation, expense_categories)
    
    # Static ratio constraints (you might retain some default values)
    problem.addConstraint(lambda s: s >= 0.2 * income, ("savings",))
    problem.addConstraint(lambda h: h <= 0.3 * income, ("housing",))
    problem.addConstraint(lambda f: f <= 0.15 * income, ("food",))
    problem.addConstraint(lambda i: i <= 0.15 * income, ("insurance",))
    
    # Dynamic constraints based on user's definition
    # Example: "savings_min" = 0.3 means savings must be at least 30% of income.
    if "savings_min" in custom_constraints:
        savings_min = custom_constraints["savings_min"]
        problem.addConstraint(lambda s: s >= savings_min * income, ("savings",))
    
    # Example: "food_max" = 0.12 means food cost must be at most 12% of income.
    if "food_max" in custom_constraints:
        food_max = custom_constraints["food_max"]
        problem.addConstraint(lambda f: f <= food_max * income, ("food",))
    
    # Additional dynamic constraints can be added similarly.
    # For example, you might let the user define a custom limit for "other" expenses:
    if "other_max" in custom_constraints:
        other_max = custom_constraints["other_max"]
        problem.addConstraint(lambda o: o <= other_max * income, ("other",))
    
    solutions = problem.getSolutions()
    return solutions


# Example usage:
if __name__ == "__main__":
    # Budget input from the user.
    budget_example = {"monthly_take_home": 4000}
    
    # Custom constraints defined by the user (e.g., through the web interface).
    # Here, the user wants savings to be at least 30% and food to be at most 12% of income.
    custom_constraints = {
        "savings_min": 0.9,  # 30% minimum savings
        "food_max": 0.12     # 12% maximum for food expenses
    }
    
    solutions = setup_csp_with_dynamic_constraints(budget_example, custom_constraints)
    
    if solutions:
        print("Number of feasible allocation solutions found:", len(solutions))
        print("Example solution:")
        print(solutions[0])
    else:
        print("No feasible allocation found with the given dynamic constraints.")


Number of feasible allocation solutions found: 495
Example solution:
{'savings': 4000, 'food': 0, 'insurance': 0, 'housing': 0, 'other': 0}
